In [1]:
!pip install llama-index
# !pip install llama-index-embeddings-langchain


In [2]:
!pip install sentence_transformers

In [3]:
!pip install llama-index-llms-ollama

In [4]:
!pip install llama-index-embeddings-ollama
!pip install llama-index-embeddings-ollama
!pip install llama-index-llms-ollama
!pip install sentence_transformers
!pip install llama-index
!pip install langchain
# !pip install langchain-experimental
# install langchain community
!pip install langchain-community
!pip install llama-index-embeddings-langchain


In [1]:
import os
from langchain.embeddings import HuggingFaceEmbeddings  # Correct class from HuggingFace
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.core import Settings, VectorStoreIndex, PromptTemplate, SimpleDirectoryReader
from llama_index.llms.ollama import Ollama  # Assuming you corrected the import here

# Loading documents from directory
docs = SimpleDirectoryReader("/mnt/c/Users/edeep/RAG/RAG_Codebase/project3_se-final_with_openai/project3_se-final_with_openai/", recursive=True).load_data()

# Load the embedding model
def load_embedding_model(model_name="sentence-transformers/all-mpnet-base-v2", device="cuda"):
    model_kwargs = {"device": device}
    encode_kwargs = {"normalize_embeddings": True}
    return HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

# Initializing components
lc_embedding_model = load_embedding_model()
embed_model = LangchainEmbedding(lc_embedding_model)
Settings.embed_model = embed_model

# Assuming local_directory and extensions are part of some other operations not shown here
local_directory = '/mnt/c/Users/edeep/RAG/RAG_Codebase'
extensions = [".py", ".ipynb", ".js", ".ts", ".md"]

# Indexing documents
index = VectorStoreIndex.from_documents(docs)

# Setting up LLM and querying capabilities
llm = Ollama(model="mistral", request_timeout=60.0)
Settings.llm = llm
query_engine = index.as_query_engine(streaming=True, similarity_top_k=4)

# Template for queries
qa_prompt_tmpl_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information above, I want you to think step by step to answer the query in a crisp manner, in case you don't know the answer say 'I don't know!'.\n"
    "Query: {query_str}\n"
    "Answer: "
)
qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)
query_engine.update_prompts({"response_synthesizer:text_qa_template": qa_prompt_tmpl})




/home/deep/miniconda3/envs/rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Perform the query
response = query_engine.query('Explain the flow in the entire codebase of how product is addded to the cart?')
print(response)

In [ ]:
response = query_engine.query('Give the important files for vendor logic in the project and explain what is  happenig in those files ?') 
print(response)

In [ ]:
response = query_engine.query('Give the important files for vendor backend logic in the project and explain what is  happenig in those files ?') 
print(response)

In [2]:
response = query_engine.query('What does product service do?') 
print(response)

 product service is responsible for handling requests related to products in the application. It provides endpoints for getting a list of products with optional filters like vendor ID, minimum price, and maximum price. It also provides endpoints for getting a specific product by its slug or ID. Furthermore, it has an endpoint for creating a new product. These functionalities are implemented in the `ProductClient` class located at `/mnt/c/Users/edeep/RAG/RAG_Codebase/project3_se-final_with_openai/project3_se-final_with_openai/frontend/application/frontend/api/ProductClient.py`.


In [3]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

# Use the PrettyPrinter to print the data
pp.pprint(response.response_txt)

(' product service is responsible for handling requests related to products in '
 'the application. It provides endpoints for getting a list of products with '
 'optional filters like vendor ID, minimum price, and maximum price. It also '
 'provides endpoints for getting a specific product by its slug or ID. '
 'Furthermore, it has an endpoint for creating a new product. These '
 'functionalities are implemented in the `ProductClient` class located at '
 '`/mnt/c/Users/edeep/RAG/RAG_Codebase/project3_se-final_with_openai/project3_se-final_with_openai/frontend/application/frontend/api/ProductClient.py`.')
